In [1]:
#############################################
#     Challenge Part 2 Locate Hotels        #
#############################################

In [2]:
# enable plotting in the Jupyter Notebook
%matplotlib inline

# Import the dependencies.
import pandas as pd
import gmaps
import gmaps.datasets
import requests

# Import the google API key.
from config import g_key

In [3]:
#############################################
#  Read City Weather Data Into a DataFrame  #
#############################################

In [4]:
# Read city weather information
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
0,Keetmanshoop,NaN,2020-05-10 10:02:46,-26.58,18.13,78.80,26,0,13.87,clear sky,0,0
1,Kapaa,US,2020-05-10 10:02:46,22.08,-159.32,73.40,78,90,14.99,overcast clouds,0,0
2,Saskylakh,RU,2020-05-10 10:02:46,71.92,114.08,25.45,96,100,13.73,overcast clouds,0,0
3,Bonthe,SL,2020-05-10 10:02:46,7.53,-12.51,82.85,79,100,9.51,overcast clouds,0,0
4,Hearst,CA,2020-05-10 10:02:46,49.68,-83.67,24.64,98,99,9.37,overcast clouds,0,0


In [5]:
#############################################
#    Select Destinations Based on Weather   #
#############################################

In [6]:
# Ask the customer the desired weather condition at the vacation destination.
destination_min_temp  = float(input("What is the minimum temperature at your vacation destination? "))
destination_max_temp  = float(input("What is the maximum temperature at your vacation destination? "))
destination_want_rain = input("Do you want it to be raining at your vacation destination? (yes/no) ")
destination_want_snow = input("Do you want it to be snowing at your vacation destination? (yes/no) ")

if destination_want_rain is None:
    destination_want_rain = "no"
    
if destination_want_snow is None:
    destination_want_snow = "no"

print()
print(f"Searching for destinations where the current temperature is between {destination_min_temp} and {destination_max_temp}")   
print(f"Show destinations where it is currently raining [{destination_want_rain}] or currently snowing [{destination_want_snow}]")

What is the minimum temperature at your vacation destination? 30
What is the maximum temperature at your vacation destination? 70
Do you want it to be raining at your vacation destination? (yes/no) no
Do you want it to be snowing at your vacation destination? (yes/no) no

Searching for destinations where the current temperature is between 30.0 and 70.0
Show destinations where it is currently raining [no] or currently snowing [no]


In [7]:
# Filter the dataset to find the cities that fit the criteria.
if ( destination_want_rain.lower() != "yes" and destination_want_snow.lower() != "yes"):
    
    print("Looking for destinations where it IS NOT raining and IS NOT snowing")
    
    preferred_cities_df = city_data_df.loc[
                                    (city_data_df["Max Temp"] <= destination_max_temp) & 
                                    (city_data_df["Max Temp"] >= destination_min_temp) & 
                                    (city_data_df["Rain Inches (last 3 hrs)"] == 0) & 
                                    (city_data_df["Snow Inches (last 3 hrs)"] == 0), :]

elif destination_want_rain.lower() == "yes":
    
    print("Looking for destinations where it IS raining")
    
    preferred_cities_df = city_data_df.loc[
                                    (city_data_df["Max Temp"] <= destination_max_temp) & 
                                    (city_data_df["Max Temp"] >= destination_min_temp) & 
                                    (city_data_df["Rain Inches (last 3 hrs)"] > 0) &
                                    (city_data_df["Snow Inches (last 3 hrs)"] == 0), :]
    
elif len(destination_want_snow) > 0 and destination_want_snow.lower() == "yes":
    
    print("Looking for destinations where it IS snowing")
    
    preferred_cities_df = city_data_df.loc[
                                    (city_data_df["Max Temp"] <= destination_max_temp) & 
                                    (city_data_df["Max Temp"] >= destination_min_temp) & 
                                    (city_data_df["Rain Inches (last 3 hrs)"] == 0) &
                                    (city_data_df["Snow Inches (last 3 hrs)"] > 0), :]


Looking for destinations where it IS NOT raining and IS NOT snowing


In [8]:
preferred_cities_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
5,Tasiilaq,GL,2020-05-10 10:02:46,65.61,-37.64,32.00,90,46,11.41,scattered clouds,0,0
6,Severo-Kurilsk,RU,2020-05-10 10:02:47,50.68,156.12,33.19,94,46,14.65,light snow,0,0
7,Ushuaia,AR,2020-05-10 10:02:47,-54.80,-68.30,39.20,93,40,2.06,scattered clouds,0,0
8,Bethel,US,2020-05-10 10:00:58,41.37,-73.41,36.00,69,75,10.29,broken clouds,0,0
9,Punta Arenas,CL,2020-05-10 10:02:47,-53.15,-70.92,42.80,93,90,2.24,overcast clouds,0,0
10,Ilulissat,GL,2020-05-10 10:02:47,69.22,-51.10,30.20,80,98,8.05,overcast clouds,0,0
11,Kaitangata,NZ,2020-05-10 10:02:47,-46.28,169.85,50.00,85,23,1.99,few clouds,0,0
12,Leningradskiy,RU,2020-05-10 10:02:47,69.38,178.42,35.40,89,100,21.00,overcast clouds,0,0
13,Airai,TL,2020-05-10 10:02:47,-8.93,125.41,60.31,100,52,3.24,broken clouds,0,0
14,Seoul,KR,2020-05-10 10:02:48,37.57,126.98,62.60,87,75,11.41,mist,0,0


In [9]:
##########################################################
# Add Hotel Name Column to Perferred Cities Data Frame   #
##########################################################

In [10]:
# Create DataFrame to store hotel names along with city, country, current weather discription, and coordinates.
preferred_cities_w_hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Add hotel name field to the data frame.
preferred_cities_w_hotel_df["Hotel Name"] = ""

preferred_cities_w_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Tasiilaq,GL,32.00,scattered clouds,65.61,-37.64,
6,Severo-Kurilsk,RU,33.19,light snow,50.68,156.12,
7,Ushuaia,AR,39.20,scattered clouds,-54.80,-68.30,
8,Bethel,US,36.00,broken clouds,41.37,-73.41,
9,Punta Arenas,CL,42.80,overcast clouds,-53.15,-70.92,
10,Ilulissat,GL,30.20,overcast clouds,69.22,-51.10,
11,Kaitangata,NZ,50.00,few clouds,-46.28,169.85,
12,Leningradskiy,RU,35.40,overcast clouds,69.38,178.42,
13,Airai,TL,60.31,broken clouds,-8.93,125.41,
14,Seoul,KR,62.60,mist,37.57,126.98,


In [11]:
##############################################
# For Selected Destination Get Nearby Hotels #
##############################################

In [12]:
# Google nearby search URL endpoint.
url_google_nearbysearch = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

hotels_found_count = 0
city_name_padding = preferred_cities_w_hotel_df['City'].str.len().max()

# Create parameter object for nearby search.
params = {
    "radius": 5000, # in meeters
    "types": "lodging", # Only looking for hotels
    "key": g_key,
    "location": "Lat:Lng"
         }



# Iterate through the preferred cities data frame
# and add hotel information.
for index, row in preferred_cities_w_hotel_df.iterrows():
    
    # Get current data frame city's latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Update the location key in the request parameters.
    params["location"] = f"{lat},{lng}"
   
    # Request nearby search for hotels.
    hotels_nearby_results = requests.get(url_google_nearbysearch, params=params).json()
 
    # Get name of the first hotel in the the nearby search result set.
    try:        
        hotel_name = hotels_nearby_results["results"][0]["name"]
        print("City: " + row['City'].ljust(city_name_padding) + "  Hotel: " + hotel_name)
        preferred_cities_w_hotel_df.loc[index, "Hotel Name"] = hotel_name
        hotels_found_count +=1
    except:
        print("City: " + row['City'].ljust(city_name_padding) + "  Hotel: *** NO HOTEL FOUND *** ... Dropping row.")
        preferred_cities_w_hotel_df.drop([index])

City: Tasiilaq              Hotel: The Red House
City: Severo-Kurilsk        Hotel: Kak Doma
City: Ushuaia               Hotel: Albatros Hotel
City: Bethel                Hotel: Courtyard by Marriott Danbury
City: Punta Arenas          Hotel: Hotel Dreams Del Estrecho
City: Ilulissat             Hotel: Hotel Arctic
City: Kaitangata            Hotel: Kaitangata Motor Camp
City: Leningradskiy         Hotel: *** NO HOTEL FOUND *** ... Dropping row.
City: Airai                 Hotel: Juvinal Martin
City: Seoul                 Hotel: LOTTE HOTEL SEOUL
City: Kodiak                Hotel: Best Western Kodiak Inn and Convention Center
City: Busselton             Hotel: Observatory Guest House
City: Khatanga              Hotel: Gostinitsa "Zapolyar'ye"
City: Victor Harbor         Hotel: Apollon Motor Inn
City: Chuy                  Hotel: Nuevo Hotel Plaza
City: Jamestown             Hotel: Hampton Inn & Suites Jamestown
City: Saint George          Hotel: Best Western Plus Abbey Inn
City: Fairba

City: Vardo                 Hotel: Vardø Hotel AS
City: Cascais               Hotel: Hotel Vila Galé Cascais
City: Neuquen               Hotel: Hotel del Comahue
City: Vestmannaeyjar        Hotel: Glamping & Camping
City: Hervey Bay            Hotel: Shelly Bay Resort
City: Sept-Iles             Hotel: Comfort Inn
City: Ancud                 Hotel: HOTEL ANCUD DON LUCAS
City: Lavrentiya            Hotel: *** NO HOTEL FOUND *** ... Dropping row.
City: Shelburne             Hotel: The Retreat at the Farm
City: Meadow Lake           Hotel: JKL Inc. R.V Park and Storage
City: Mar Del Plata         Hotel: Gran Hotel Mar del Plata habitaciones
City: Port Macquarie        Hotel: Rydges Port Macquarie
City: Santa Maria           Hotel: Hotel Morotin
City: Ust-Nera              Hotel: Hotel Ust-Nera
City: Husavik               Hotel: Bjarnabúð
City: College               Hotel: Pike's Waterfront Lodge
City: Lasa                  Hotel: Pension Feldgärtenhof
City: Kuri                  Hotel: Ca

In [13]:
# Reset index to renumber the data frame rows indexes sequencially.
#
# Rows in the data frame were deleted when no nearby hotel 
# was found for the City name for the current data frame row.
# The deleted row created gaps in the sequencial index numbering
# of rows in the data frame. To create a sequencially numbered index
# for all remaining row in the data frame, the current existing
# index was dropped and the data frame index was reset. 

preferred_cities_w_hotel_reset_df = preferred_cities_w_hotel_df.reset_index(drop=True)
preferred_cities_w_hotel_reset_df.head(30)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Tasiilaq,GL,32.00,scattered clouds,65.61,-37.64,The Red House
1,Severo-Kurilsk,RU,33.19,light snow,50.68,156.12,Kak Doma
2,Ushuaia,AR,39.20,scattered clouds,-54.80,-68.30,Albatros Hotel
3,Bethel,US,36.00,broken clouds,41.37,-73.41,Courtyard by Marriott Danbury
4,Punta Arenas,CL,42.80,overcast clouds,-53.15,-70.92,Hotel Dreams Del Estrecho
5,Ilulissat,GL,30.20,overcast clouds,69.22,-51.10,Hotel Arctic
6,Kaitangata,NZ,50.00,few clouds,-46.28,169.85,Kaitangata Motor Camp
7,Leningradskiy,RU,35.40,overcast clouds,69.38,178.42,
8,Airai,TL,60.31,broken clouds,-8.93,125.41,Juvinal Martin
9,Seoul,KR,62.60,mist,37.57,126.98,LOTTE HOTEL SEOUL


In [20]:
# Save the perferred cities with nearby hotels information to csv file without the inxex column
preferred_cities_w_hotel_df.to_csv("weather_data/WeatherPy_vacation.csv", index=False)

In [15]:
###########################################################
# Plot Perferred Cities With Nearby Hotels on Google Maps #
###########################################################

In [19]:
# Popup markers template for destination cities with hotel and weather information
destination_city_info_box_template = """
<dl>
    <dt>Hotel Name</dt>
        <dd>{Hotel Name}</dd>
    <dt>City</dt>
        <dd>{City}</dd>
    <dt>Country</dt>
        <dd>{Country}</dd>
    <dt>Current Weather</dt>
        <dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Create google map pop marker info boxes for destination cities.
destination_city_info_boxes = [destination_city_info_box_template.format(**row) for index, row in preferred_cities_w_hotel_df.iterrows()]

# Get destination cities locations
destination_city_locations = preferred_cities_w_hotel_df[["Lat", "Lng"]]

# # Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

# Add a pop-up marker for each city.
marker_layer = gmaps.marker_layer(destination_city_locations, info_box_content=destination_city_info_boxes)

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))